In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import numba
from numba import float64 as f64
import ipywidgets as widgets
from ipywidgets import FloatSlider, FloatLogSlider, IntSlider,HBox, VBox
import matplotlib.ticker as mtick

In [2]:
@numba.vectorize
def nb_clip(x, a, b):
    # numba doesn't support np.clip yet...
    if   x > b: return b
    elif x < a: return a
    else:       return x

# ===============================================
# Define the MBBEFD distribution, without class
# ===============================================

##remove class
#replace X1,X2 with c and mpl values (SI already included) and replace functions of X1,X2 with new functions (for ilf,mpl and mean)

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_cdf(x, b, g, mpl):
    # TODO: test all cases...
    if (np.isnan(x) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan
    x /= mpl
    if   x>=1:                    return 1
    elif g==1 or b==0 or x==0:    return 0
    elif b==1 and g>1:            return 1 - 1/(1+(g-1)*x)
    elif (b*g)==1 and g>1:        return 1 - b**x 
    elif b>0 and b and (b*g)!=1 and g>1: return 1 - (1 - b) / ((g - 1) * (b ** (1 - x)) + (1 - g * b))
    else: return np.nan

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_ppf(q, b, g, mpl):
    # TODO: test all cases...
    
    if (np.isnan(q) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan

    if q <= 0: return 0
    if q >= 1: return mpl # TODO: check
    return mpl * (1 - np.log( ((1-b)/(1-q) - 1 + g*b) / (g-1) ) / np.log(b))

@numba.vectorize([f64(f64, f64, f64, f64)])
def _mbbefd_ilf(x, b, g, mpl):
    # TODO: test all cases...
    
    if (np.isnan(x) or np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan

    if x > mpl: return 0 # TODO: check
    res = np.log(((g-1)*b + (1 - g*b) * (b**(x/mpl))) / (1 - b)) / np.log(g*b)
    return nb_clip(res, 0, 1) # TODO: check, mpl?\

@numba.vectorize([f64(f64, f64, f64)])
def _mbbefd_mean(b, g, mpl):
    # TODO: test
    if (np.isnan(b) or np.isnan(g) or np.isnan(mpl)) or not (b >= 0 and g >= 1 and mpl >= 0):
        return np.nan
    return mpl * (np.log(g * b) * (1 - b) / np.log(b) / (1 - g * b))


@numba.njit
def MBBEFD_cdf(x,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_cdf(x, b, g, mpl)

@numba.njit
def MBBEFD_ilf(x,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_ilf(x, b, g, mpl)

@numba.njit
def MBBEFD_ppf(q,c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_ppf(q, b, g, mpl)

@numba.njit
def MBBEFD_mean(c,mpl):
    b,g=MBBEFD_c(c)
    return _mbbefd_mean(b, g, mpl)

@numba.njit
def MBBEFD_c(c):
    b = np.exp(3.1 - 0.15 * (1 + c) * c)
    g = np.exp((0.78 + 0.12 * c) * c)
    return b,g

In [3]:
#testing
# @widgets.interact(c=IntSlider(description='c', value=3, min=1, max=15, step=1))  
# def getbg(c):
#     c

interactive(children=(IntSlider(value=3, description='c', max=15, min=1), Output()), _dom_classes=('widget-int…

In [4]:
@widgets.interact(b = FloatSlider(description='b', value=5, min=1.0001, max=100, step=.1),
                  g = FloatLogSlider(description='g',value=10, base=10,
                  min=0, # min exponent of base
                  max=3, # max exponent of base,
                  step=.1))
def f(b, g):
    x = np.linspace(0, 1, 100)
    f,(ax1, ax2) = plt.subplots(2,1, figsize=(12, 10), constrained_layout=True, sharex=True)
    f.suptitle('MBBEFD general form (2 parameter) distributions for b,g = ' f' b={b:.5f} g={g:.5f}', fontsize=16)
    ax1.plot(x, _mbbefd_cdf(x,b,g,1), linewidth=1.5)
    ax2.plot(x, _mbbefd_ilf(x,b,g,1), linewidth=1.5)
    
    ax1.set_title('cdf')
    
    ax1.set_ylabel('cumulative probability')
    ax2.set_title('ilf (exposure curve)')
    ax2.set_xlabel('damage factor')
    ax2.set_ylabel('premium share (of fgu premium)')
   
    for c in np.array([5,4,3,2,1]):
        y = MBBEFD_cdf(x,c,1)
        ax1.plot(x, y, linestyle='dashed', linewidth=1, color=(c/6,c/6,c/6))
        b,g=MBBEFD_c(c)
        y = MBBEFD_ilf(x,c,1)
        ax2.plot(x, y, linestyle='dashed', linewidth=1, color=(c/6,c/6,c/6),label=f'c={c} b={b:.5f} g={g:.5f}')
        ax2.legend();
        
    ax1.grid(True)
    ax2.grid(True)
    ax1.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
    ax1.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
    ax2.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
    ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))

NameError: name 'c' is not defined